In [1]:
from keras.datasets import mnist
from keras.datasets import cifar10
from HSIC import hsic_gam
import numpy as np
import math
import random

nc = 32 # no of categories
df = 256 // nc # dividing factor

Using TensorFlow backend.


In [2]:
# Identical test for MNIST taking all pixels across all images in the dataset

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
catcount = np.zeros(nc)
train_images = train_images // df
for i in range(nc):
    catcount[i] = np.count_nonzero(train_images == i)
# print(catcount) # no of pixels in the ith category across the dataset, 0<=i<=15
e = train_images.shape[0]*train_images.shape[1]*train_images.shape[2] / nc # expected no in each category if identically distributed
qmnist = np.sum((catcount - e)**2) / e # chi squared statistic
print(qmnist)

959599004.3099864


In [3]:
# Identical test for CIFAR10 taking all pixels across all images in the dataset

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = np.transpose(train_images) // df
catcount = np.zeros((3,nc))
for i in range(3):
    for j in range(nc):
        catcount[i][j] = np.count_nonzero(train_images[i] == j)

e = train_images.shape[1]*train_images.shape[2]*train_images.shape[3] // nc

qR = np.sum((catcount[0] - e)**2) / e
qG = np.sum((catcount[1] - e)**2) / e
qB = np.sum((catcount[2] - e)**2) / e
print(qR, qG, qB)

5772588.1073 6792260.36422875 5573899.66601375


In [4]:
# Comparing chi squared statistic for the 2 datasets

qavg = (qR + qG + qB)/3
print(qavg)
print(qmnist/qavg)

6046249.379180834
158.70979579740657


In [5]:
# Independent test for MNIST, taking all pixels of a single image

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

no_of_images = 100
image_indexes = random.sample(list(range(0, test_images.shape[0])), no_of_images)

def neighbour_pixel(x,y,k):
    pixval = 0
    count = 0
    for i in range(-1,2):
        for j in range(-1,2):
            if (not (i==0 and j==0)) and x+i>=0 and x+i<28 and y+j>=0 and y+j<28:
                count+=1
                pixval+=train_images[k][x+i][y+j]
    pixval=pixval/count
    return pixval

minr= 1
maxr= 0
avg=0
train_nbs = np.zeros((28,28))

for k in image_indexes:
    for i in range(28):
        for j in range(28):
            train_nbs[i][j] = neighbour_pixel(i,j,k)
    Xbar = np.mean(train_images[k])
    Ybar = np.mean(train_nbs)
    num = np.sum(np.multiply(train_images[k]-Xbar, train_nbs-Ybar))
    denom1 = np.sum((train_images[k]-Xbar)**2)
    denom2 = np.sum((train_nbs-Ybar)**2)
    r = num / math.sqrt(denom1*denom2)
    if r<minr:
        minr = r
    if r>maxr:
        maxr = r
    avg = avg+r
avg/=no_of_images
print("Correlation coefficient for", no_of_images, "images:")
print("min=",minr)
print("max=",maxr)
print("avg=",avg)

Correlation coefficient for 100 images:
min= 0.7260655629923657
max= 0.9635609377302125
avg= 0.9172471227946751


In [6]:
# Independent test for CIFAR10, taking all pixels of a single image

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

no_of_images = 100
image_indexes = random.sample(list(range(0, train_images.shape[0])), no_of_images)

train_trans = np.zeros((3,32,32))
train_nbs = np.zeros((32,32))
minr=[1,1,1]
maxr = np.zeros(3)
avgr = np.zeros(3)

def neighbour_cifar(x,y,c):
    pixval = 0
    count = 0
    for i in range(-1,2):
        for j in range(-1,2):
            if (not (i==0 and j==0)) and x+i>=0 and x+i<32 and y+j>=0 and y+j<32:
                count+=1
                pixval+=train_trans[c][x+i][y+j]
    pixval=pixval/count
    return pixval

for k in image_indexes:
    train_trans = np.transpose(train_images[k])
    for c in range(3):
        for i in range(32):
            for j in range(32):
                train_nbs[i][j] = neighbour_cifar(i,j,c)
        Xbar = np.mean(train_trans[c])
        Ybar = np.mean(train_nbs)
        num = np.sum(np.multiply(train_trans[c]-Xbar, train_nbs-Ybar))
        denom1 = np.sum((train_trans[c]-Xbar)**2)
        denom2 = np.sum((train_nbs-Ybar)**2)
        r = num / math.sqrt(denom1*denom2)
        if r < minr[c]:
            minr[c] = r
        if r > maxr[c]:
            maxr[c] = r
        avgr[c] = avgr[c] + r

avgr /= no_of_images
print("Correlation coefficient for", no_of_images, "images:")
for i in range(3):
    print("\nChannel ", i)
    print("min=",minr[i])
    print("max=",maxr[i])
    print("avg=",avgr[i])

Correlation coefficient for 100 images:

Channel  0
min= 0.7972454199403878
max= 0.9952347673058268
avg= 0.951680783617381

Channel  1
min= 0.7543336122981747
max= 0.9953640563943111
avg= 0.9497018628335505

Channel  2
min= 0.5851802733428113
max= 0.9955651220491615
avg= 0.9494368961013763


In [7]:
# Trying HS test

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_trans = np.zeros((3,32,32))
train_nbs = np.zeros((32,32))
no_of_images = 100

for k in range(no_of_images):
    train_trans = np.transpose(train_images[k])
    for c in range(3):
        for i in range(32):
            for j in range(32):
                train_nbs[i][j] = neighbour_cifar(i,j,c)
        x = train_trans[c].reshape((32*32,1))
        print(x.shape)
        y = train_nbs.reshape((32*32,1))
        testStat, thresh = hsic_gam(x, y, alph = 0.05)
        print(testStat, thresh)

(1024, 1)


/home/vishal/HSIC.py:39: RuntimeWarning: overflow encountered in exp
  H = np.exp(-H/2/(deg**2))
/home/vishal/HSIC.py:97: RuntimeWarning: invalid value encountered in subtract
  K = K - np.diag(np.diag(K))


nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1024, 1)
nan nan
(1

In [8]:
# Independent test for MNIST, taking same pixel across all images

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

no_of_pixels = 50

a = random.sample(list(range(0,28*28)), no_of_pixels)
pixels = [(n//28, n%28) for n in a]

minr= 1
maxr= 0
avg=0
X = np.zeros((train_images.shape[0]))
Y = np.zeros((train_images.shape[0]))

for (i,j) in pixels:
    for k in range(train_images.shape[0]):
        X[k] = train_images[k][i][j]
        Y[k] = neighbour_pixel(i,j,k)
    Xbar = np.mean(X)
    Ybar = np.mean(Y)
    num = np.dot(X-Xbar, Y-Ybar)
    denom1 = np.sum((X-Xbar)**2)
    denom2 = np.sum((Y-Ybar)**2)
    if denom1==0 or denom2==0:
        no_of_pixels =no_of_pixels - 1
        continue
    r = num / math.sqrt(denom1*denom2)
    print(r)
    if r<minr:
        minr = r
    if r>maxr:
        maxr = r
    avg = avg+r
avg/=no_of_pixels
print("Correlation coefficient for", no_of_pixels, "pixels:")
print("min=",minr)
print("max=",maxr)
print("avg=",avg)

0.7597468942723918
0.6453499246753217
0.38731027981047667
0.907773474569046
0.9175834963475539
0.902470670205603
0.917219588698478
0.9070644565945929
0.8854759125063321
0.8605509697109249
0.9099350147358045
0.6998327731527538
0.0160164702846705
0.4195403841168866
0.69285713656212
0.7217432076385978
0.6019051903391301
0.28337568086455767
0.5944431571424798
0.8835606260836996
0.7085719207879961
0.89914774227006
0.6431986735178845
0.4513258543276867
0.8864653828801339
0.757977110873341
0.7571863147181693
0.5406731735551681
0.1536971978005766
0.9324177058538112
0.8508767268479849
0.9128394408209797
0.5349132150983239
0.4196297681730535
0.787174761979205
0.9250709221176574
0.46168897583208746
0.7014556918906598
0.6663043097690812
0.6823384795286873
0.9187834408931882
0.8243137363312413
0.07813222651993924
0.895563394838705
0.5583826897184754
Correlation coefficient for 45 pixels:
min= 0.0160164702846705
max= 0.9324177058538112
avg= 0.6858196481167893


In [9]:
# Independent test for CIFAR10, taking same pixel across all images

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

no_of_pixels = 20

a = random.sample(list(range(0,32*32)), no_of_pixels)
pixels = [(n//32, n%32) for n in a]

minr= [1,1,1]
maxr= np.zeros(3)
avg= np.zeros(3)
X = np.zeros((train_images.shape[0]))
Y = np.zeros((train_images.shape[0]))

def nb_cifar(k,x,y,c):
    pixval = 0
    count = 0
    for i in range(-1,2):
        for j in range(-1,2):
            if (not (i==0 and j==0)) and x+i>=0 and x+i<32 and y+j>=0 and y+j<32:
                count+=1
                pixval+=train_images[k][x+i][y+j][c]
    pixval=pixval/count
    return pixval

for (i,j) in pixels:
    for c in range(3):
        for k in range(train_images.shape[0]):
            X[k] = train_images[k][i][j][c]
            Y[k] = nb_cifar(k,i,j,c)
        Xbar = np.mean(X)
        Ybar = np.mean(Y)
        num = np.sum(np.multiply(X-Xbar,Y-Ybar))
        denom1 = np.sum((X-Xbar)**2)
        denom2 = np.sum((Y-Ybar)**2)
        if denom1==0 or denom2==0:
            no_of_pixels =no_of_pixels - 1
            continue
        r = num / math.sqrt(denom1*denom2)
        if r<minr[c]:
            minr[c] = r
        if r>maxr[c]:
            maxr[c] = r
        avg[c] = avg[c]+r
avg/=no_of_pixels
print("Correlation coefficient for", no_of_pixels, "pixels:")
for i in range(3):
    print("\nChannel ", i)
    print("min=",minr[i])
    print("max=",maxr[i])
    print("avg=",avg[i])

Correlation coefficient for 20 pixels:

Channel  0
min= 0.9525172862664026
max= 0.9849013684098377
avg= 0.9624088681009619

Channel  1
min= 0.9504571109363242
max= 0.9850045820045451
avg= 0.9614586563620187

Channel  2
min= 0.9537933609712125
max= 0.9876731297872245
avg= 0.9659191126009201
